1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [278]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import re

In [279]:
def comp_distribution(text = ''):
    res = {'min_compensation': None, 'max_compensation': None, 'currency': None}
    if text == '':
        return res
    text = re.sub(r'[\.\s]', "", text)
    text = re.split('(\d+)', text)
    res['currency'] = text.pop()
    if len(text) == 4:
        res['min_compensation'] = int(text[1])
        res['max_compensation'] = int(text[3])
    if len(text) == 2:
        if text[0].lower() == 'от':
            res['min_compensation'] = int(text[1])
        if text[0].lower() == 'до':
            res['max_compensation'] = int(text[1])
    return res

#print(comp_distribution('от 4 000 USD	'))

In [280]:
columns = ['Position_name', 'Employer_name', 'City', 'Position_link', 'Source', 'Min_compensation',
           'Max_compensation', 'Currency']
df = pd.DataFrame(columns=columns)

In [281]:
main_link = 'https://hh.ru'

header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
          'Accept':'*/*'}

params = {'L_save_area': 'true',
          'clusters': 'true',
          'enable_snippets': 'true',
          'text': 'Python',
          'showClusters': 'true'}

In [282]:
response = requests.get(main_link + '/search/vacancy', headers=header, params=params).text
soup = bs(response, 'lxml')

In [283]:
page =0
while True:
    
    vacancy_block = soup.find('div', {'class': 'vacancy-serp'})

    vacancy_list_premium = vacancy_block.find_all('div', {'data-qa': 'vacancy-serp__vacancy vacancy-serp__vacancy_premium'})
    for vacancy in vacancy_list_premium:
        vacancy_data = []
        vacancy_data.append(vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'}).getText())
        vacancy_data.append(vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'}).getText())
        vacancy_data.append(vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText())
        vacancy_data.append(vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href'])
        vacancy_data.append(main_link)
        compensation_data = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
        compensation_data = comp_distribution(compensation_data)
        vacancy_data.append(compensation_data['min_compensation'])
        vacancy_data.append(compensation_data['max_compensation'])
        vacancy_data.append(compensation_data['currency'])
        df = df.append(pd.Series(vacancy_data, index=df.columns), ignore_index=True)

    vacancy_list = vacancy_block.find_all('div', {'data-qa': 'vacancy-serp__vacancy'})
    for vacancy in vacancy_list:
        vacancy_data = []
        vacancy_data.append(vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'}).getText())
        employer_name = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'})
        if employer_name:                        
            vacancy_data.append(employer_name.getText())
        else:
            vacancy_data.append(None)
        vacancy_data.append(vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText())
        vacancy_data.append(vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href'])
        vacancy_data.append(main_link)
        compensation_data = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
        compensation_data = comp_distribution(compensation_data)
        vacancy_data.append(compensation_data['min_compensation'])
        vacancy_data.append(compensation_data['max_compensation'])
        vacancy_data.append(compensation_data['currency'])
        df = df.append(pd.Series(vacancy_data, index=df.columns), ignore_index=True)
    
    page += 1
    next_page_link_block = soup.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})

    if not next_page_link_block:
        break

    next_page_link = next_page_link_block['href']
    response = requests.get(main_link + next_page_link, headers=header).text
    soup = bs(response, 'lxml')
    
print(page)

40


In [284]:
df.head(50)

,Position_name,Employer_name,City,Position_link,Source,Min_compensation,Max_compensation,Currency
0,Python / Django fullstack программист,ООО Чистый Маркетинг,Москва,https://hh.ru/vacancy/37461514?query=Python,https://hh.ru,110000,180000,руб
1,Скилловый Python Developer на бэкенд,Интеллектуальные системы,"Санкт-Петербург, Петроградская",https://hh.ru/vacancy/37765447?query=Python,https://hh.ru,250000,None,руб
2,Программист-разработчик python (midle),CATAPULTO.RU,Москва,https://hh.ru/vacancy/37674249?query=Python,https://hh.ru,100000,180000,руб
3,AI Scientist (R&D),Oxagile,"Минск, Михалово",https://hh.ru/vacancy/37281217?query=Python,https://hh.ru,None,None,None
4,"Backend разработчик Python (Middle, Senior)",Infatica Pte Ltd,Москва,https://hh.ru/vacancy/37742512?query=Python,https://hh.ru,80000,120000,руб
5,Researcher in Mathematical Modeling and Algori...,ООО СК хайникс мемори солюшнс Восточная Европа,"Минск, Немига",https://hh.ru/vacancy/35441422?query=Python,https://hh.ru,None,None,None
6,Специалист по углубленной аналитике данных (Da...,"А1. Информационные технологии, телеком","Минск, Октябрьская и еще 1",https://hh.ru/vacancy/37732722?query=Python,https://hh.ru,None,None,None
7,Frontend-разработчик midle (React.js \ ExtJS),CATAPULTO.RU,Москва,https://hh.ru/vacancy/37328608?query=Python,https://hh.ru,100000,200000,руб
8,Senior Data Engineer / Python,ООО IHS Global,"Минск, Немига",https://hh.ru/vacancy/37762953?query=Python,https://hh.ru,None,None,None
9,Senior Python Developer,УП Руна Системс,Минск,https://hh.ru/vacancy/35095020?query=Python,https://hh.ru,None,None,None


In [285]:
df.describe()

,Position_name,Employer_name,City,Position_link,Source,Min_compensation,Max_compensation,Currency
count,2000,1997,2000,2000,2000,591,385,660
unique,1291,934,316,1769,1,76,61,7
top,DevOps Engineer,Тинькофф,Москва,https://hh.ru/vacancy/35958348?query=Python,https://hh.ru,100000,150000,руб
freq,25,41,440,3,2000,52,52,583
